In [23]:
import tensorflow as tf
import pandas as pd
import numpy as np
import wandb

# Tensorflow

#### Config

In [24]:
import configs
config = configs.config_1

### WandB connection

In [25]:

# wandb.login()
# wandb.init(project="assignment-1", name="tensorflow", reinit=True)

### Loading the data

In [26]:
x_train = pd.read_csv('data/train/data.csv').to_numpy(dtype='float32')
y_train = pd.read_csv('data/train/labels.csv').to_numpy(dtype='float32')

x_val = pd.read_csv('data/val/data.csv').to_numpy(dtype='float32')
y_val = pd.read_csv('data/val/labels.csv').to_numpy(dtype='float32')

print(x_train[0])
print(y_train[0])
print()
print(x_val[0])
print(y_val[0])

[2.         0.33221272 1.         1.         3.5115454 ]
[1.]

[ 3.        -0.6710254  0.         0.         2.180892 ]
[0.]


### Building a model

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(5,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(rate=config['dropout_rate']),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dropout(rate=config['dropout_rate']),
    tf.keras.layers.Dense(1, activation='sigmoid'),    
])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 16)                96        
                                                                 
 dropout_4 (Dropout)         (None, 16)                0         
                                                                 
 dense_7 (Dense)             (None, 8)                 136       
                                                                 
 dropout_5 (Dropout)         (None, 8)                 0         
                                                                 
 dense_8 (Dense)             (None, 1)                 9         
                                                                 
Total params: 241
Trainable params: 241
Non-trainable params: 0
_________________________________________________________________


In [29]:
if config['optimizer'] == 'SGD':
    optimizer = tf.keras.optimizers.SGD(learning_rate=config['learning_rate'])
elif config['optimizer'] == 'Adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=config['learning_rate'])

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [30]:
def lr_scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        #return 1 / (1 + config['decay_rate'] * epoch) * config['learning_rate']\n",
        return (0.99 ** epoch) * config['learning_rate']

scheduler_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

### Training the model

In [31]:
history=model.fit(
    x_train,
    y_train,
    epochs=config['epochs'],
    validation_data=(x_val, y_val),
    batch_size=config['batch_size'],
    shuffle=True,
    callbacks=[
        scheduler_callback,
        #wandb.keras.WandbCallback()
    ]
)

Epoch 1/100
20/20 [==============================] - 1s 18ms/step - loss: 1.0236 - accuracy: 0.3857 - val_loss: 0.7928 - val_accuracy: 0.3774 - lr: 0.0100
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7292 - accuracy: 0.4003 - val_loss: 0.7016 - val_accuracy: 0.4000 - lr: 0.0100
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6793 - accuracy: 0.6094 - val_loss: 0.6773 - val_accuracy: 0.6415 - lr: 0.0100
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6624 - accuracy: 0.6661 - val_loss: 0.6639 - val_accuracy: 0.6340 - lr: 0.0100
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6515 - accuracy: 0.6645 - val_loss: 0.6567 - val_accuracy: 0.6264 - lr: 0.0100
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6437 - accuracy: 0.6613 - val_loss: 0.6506 - val_accuracy: 0.6302 - lr: 0.0100
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.

In [32]:
import plotly.express as px
fig = px.line({'loss': history.history['loss'], 'val_loss': history.history['val_loss']})
fig.show()

In [33]:
import plotly.express as px
fig = px.line({'loss': history.history['accuracy'], 'val_loss': history.history['val_accuracy']})
fig.show()